In [ ]:
# !pip install -U openai-whisper
# -------------OR----------------
!pip install git+https://github.com/openai/whisper.git

# on Ubuntu or Debian
!sudo apt update && sudo apt install ffmpeg

!pip install pydub


  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-qe20a9mw
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-qe20a9mw
  Resolved https://github.com/openai/whisper.git to commit c0d2f624c09dc18e709e37c2ad90c039a4eb72a2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 84.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 71.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 19.5 MB/s eta 0:00:00
   ━━━━

In [ ]:
import os
import shutil
import whisper

model_name = "medium"
cache_path = os.path.expanduser(f"~/.cache/whisper/{model_name}.pt")
custom_model_dir = "./whisper_model_dir"
custom_model_path = os.path.join(custom_model_dir, f"{model_name}.pt")

# 1. Download model if not already cached
if not os.path.exists(cache_path):
    print(f"Downloading {model_name} model...")
    model = whisper.load_model(model_name)
else:
    print(f"{model_name}.pt already cached.")

# 2. Copy to custom folder
os.makedirs(custom_model_dir, exist_ok=True)

if not os.path.exists(custom_model_path):
    shutil.copy(cache_path, custom_model_path)
    print(f"Model copied to: {custom_model_path}")
else:
    print("Model already exists in custom folder.")


medium.pt already cached.
Model already exists in custom folder.


In [ ]:
import os
from pydub import AudioSegment
from pydub.utils import make_chunks
import whisper

In [ ]:

def transcribe_chunk(chunk, chunk_number, volume_boost_db=10):
    chunk = chunk + volume_boost_db
    chunk_name = f"temp_chunk_{chunk_number}.wav"
    chunk.export(chunk_name, format="wav")


    result = model.transcribe(chunk_name,language="en")  # ✅ handles full chunk automatically
    os.remove(chunk_name)
    return result["text"]

In [ ]:
def transcribe_long_audio(file_path, chunk_length_ms=120000):  # 2-minute chunks
    audio = AudioSegment.from_mp3(file_path)
    chunks = make_chunks(audio, chunk_length_ms)

    full_transcript = ""
    for i, chunk in enumerate(chunks):
        print(f"Transcribing chunk {i+1} of {len(chunks)}...")
        chunk_transcript = transcribe_chunk(chunk, i)
        full_transcript += chunk_transcript + " "

    return full_transcript.strip()

In [ ]:
# Usage
mp3_file_path = "/content/case_study.mp3"
transcription = transcribe_long_audio(mp3_file_path)
print(transcription)

# Save the transcription
with open("transcription.txt", "w") as f:
    f.write(transcription)

Transcribing chunk 1 of 7...
Transcribing chunk 2 of 7...
Transcribing chunk 3 of 7...
Transcribing chunk 4 of 7...
Transcribing chunk 5 of 7...
Transcribing chunk 6 of 7...
Transcribing chunk 7 of 7...
Hi Lucy, nice to meet you. So I understand that you were referred here by your GP because you've been feeling quite down recently. Yeah, I thought I should, I've been feeling quite bad for quite a while so I thought maybe it's time to see someone about it because I don't want to feel like this anymore. Okay, so from your notes I saw that you saw the GP about three months ago. Yeah. So how long have you been feeling down altogether? Quite a few months before that as well actually, but it's gotten pretty bad these past few months so that's kind of why I came. Okay, so has it got worse since you saw your GP? Yeah. Okay, so tell me how are things at the moment? Well I kind of started just feeling a bit low and wound up about things but I don't know, just recently I just can't really be both